In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_theme(style="whitegrid")


In [3]:
#Loading the assam files
df_enrolment = pd.read_csv('/content/drive/MyDrive/Data_Hackathon/Andaman and Nicobar /andaman_nicobar_enrollment.csv')
df_demographic = pd.read_csv('/content/drive/MyDrive/Data_Hackathon/Andaman and Nicobar /andaman_nicobar_demographic.csv')
df_biometric = pd.read_csv('/content/drive/MyDrive/Data_Hackathon/Andaman and Nicobar /andaman_nicobar_biomatric.csv')

df_enrolment.head()

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater
0,01-09-2025,Andaman and Nicobar Islands,Nicobar,744304,1,0,0
1,01-09-2025,Andaman and Nicobar Islands,South Andaman,744107,1,0,0
2,04-09-2025,Andaman and Nicobar Islands,North And Middle Andaman,744205,1,0,0
3,09-09-2025,Andaman and Nicobar Islands,Nicobar,744301,0,2,0
4,09-09-2025,Andaman and Nicobar Islands,North And Middle Andaman,744209,2,0,0


In [ ]:
df_enrolment.head()

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater
0,01-09-2025,Maharashtra,Ahmadnagar,413701,1,2,1
1,01-09-2025,Maharashtra,Ahmadnagar,413703,1,1,0
2,01-09-2025,Maharashtra,Ahmadnagar,413705,3,0,0
3,01-09-2025,Maharashtra,Ahmadnagar,414002,2,1,0
4,01-09-2025,Maharashtra,Ahmadnagar,414006,2,0,0


In [4]:
# We create a function so we don't have to repeat code for all 3 files
def clean_aadhaar_file(df):
    # Fix the date
    df['date'] = pd.to_datetime(df['date'], dayfirst=True)

    # Clean up text (Remove spaces and make it 'Guwahati' style)
    df['state'] = df['state'].astype(str).str.strip().str.title()
    df['district'] = df['district'].astype(str).str.strip().str.title()

    # Ensure Pincode is an integer (to avoid matching '781001' with 781001)
    df['pincode'] = df['pincode'].astype(int)

    # Group and Sum: This ensures we have 1 row per Pincode per Day
    # This is the "Aggregation" part of your member task
    numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
    if 'pincode' in numeric_cols: numeric_cols.remove('pincode')

    df_cleaned = df.groupby(['date', 'state', 'district', 'pincode'])[numeric_cols].sum().reset_index()
    return df_cleaned

# 1. Create a function to remove ALL junk except letters and spaces
import re
def ultra_clean_names(text):
    text = str(text).strip().title()
    # Remove everything that isn't a letter or a space
    return re.sub(r'[^a-zA-Z\s]', '', text)

# 2. Apply this to your raw data BEFORE the main cleaning function
for df in [df_enrolment, df_demographic, df_biometric]:
    df['district'] = df['district'].apply(ultra_clean_names)

#applying cleaning to all three
df_enrolment_c = clean_aadhaar_file(df_enrolment)
df_demographic_c = clean_aadhaar_file(df_demographic)
df_biometric_c = clean_aadhaar_file(df_biometric)


In [5]:
raw_all = pd.concat([
    df_enrolment['district'],
    df_demographic['district'],
    df_biometric['district']
])

raw_districts_cleaned = raw_all.apply(ultra_clean_names).unique()
raw_count = len(raw_districts_cleaned)

print(f"District in Raw (Standardized): {raw_count}")

District in Raw (Standardized): 5


In [18]:
#THE MASTER MERGE (outer join)
#we use outer so if any pinccode has only enrollment but no updates we don't lose it

master = pd.merge(df_enrolment_c, df_demographic_c, on=['date','state','district','pincode'], how = 'outer')
master = pd.merge(master, df_biometric_c, on=['date','state','district','pincode'], how = 'outer')

#HANDLING Missing Values
master = master.fillna(0)

In [20]:
# 1. Create the specific mapping for the Islands
an_fixes = {
    'Andamans': 'South Andaman', # Usually refers to the main cluster
    'Nicobars': 'Nicobar'
}

# 2. Apply the fixes
master['district'] = master['district'].replace(an_fixes)

In [21]:
# This lists all unique districts alphabetically
all_districts_c = sorted(master['district'].unique())
print(f"Total Unique Districts: {len(all_districts_c)}")
for d in all_districts_c:
    print(d)

Total Unique Districts: 3
Nicobar
North And Middle Andaman
South Andaman


In [22]:
duplicate_rows = master[master.duplicated(subset=['date','district','pincode'])]
print(f"Total Duplicate Rows: {len(duplicate_rows)}")

Total Duplicate Rows: 573


In [24]:
# This finds the rows that are causing the '438' count
trouble_rows = master[master.duplicated(subset=['date', 'pincode'], keep=False)]

# Let's look at the first two 'duplicates' side-by-side
if not trouble_rows.empty:
    print("--- DIAGNOSTIC DATA ---")
    # We use .to_dict() to see hidden spaces or tiny decimal differences
    print(trouble_rows.head(2).to_dict('records'))
else:
    print("No duplicates found by Date/Pincode. Check if District names are the difference.")

--- DIAGNOSTIC DATA ---
[{'date': Timestamp('2025-06-03 00:00:00'), 'state': 'Andaman And Nicobar Islands', 'district': 'North And Middle Andaman', 'pincode': 744211, 'age_0_5': 0.0, 'age_5_17': 0.0, 'age_18_greater': 0.0, 'demo_age_5_17': 0.0, 'demo_age_17_': 0.0, 'bio_age_5_17': 0.0, 'bio_age_17_': 1.0}, {'date': Timestamp('2025-06-03 00:00:00'), 'state': 'Andaman And Nicobar Islands', 'district': 'South Andaman', 'pincode': 744211, 'age_0_5': 0.0, 'age_5_17': 0.0, 'age_18_greater': 0.0, 'demo_age_5_17': 0.0, 'demo_age_17_': 0.0, 'bio_age_5_17': 0.0, 'bio_age_17_': 2.0}]


In [ ]:
#ADDED FEATURES
master['total_enrolments'] = master['age_0_5'] + master['age_5_17'] + master['age_18_greater']
master['total_updates'] = master['demo_age_5_17'] + master['demo_age_17_'] + master['bio_age_5_17'] + master['bio_age_17_']

**VERIFICATION PART OF THE DATASET!!**

In [ ]:
#Check SUM
#total Enrolment in raw files
raw_total = df_enrolment['age_0_5'].sum() + df_enrolment['age_5_17'].sum() + df_enrolment['age_18_greater'].sum()

#Calculate Total enrolment in CLEANED Master file
cleaned_total = master['total_enrolments'].sum()

print(f"Total Enrolment in Raw Files: {raw_total}")
print(f"Total Enrolment (Cleaned): {cleaned_total}")

if raw_total == cleaned_total:
  print("Verification Successful")
else:
  print("Verification Failed")

Total Enrolment in Raw Files: 445029
Total Enrolment (Cleaned): 445029.0
Verification Successful


In [25]:
#HANDLING DUPLICATE VALUES
is_unique = master.duplicated(subset=['date','district','pincode']).any()

if not is_unique:
  print("Verification Successful: NO Duplicate values")
else:
  print("Verification Failed: Duplicate Values are present")

Verification Failed: Duplicate Values are present


In [ ]:
#NULL Value Check
null_count = master.isnull().sum().sum()

if null_count.sum() == 0:
  print("Null Value Verification Successfull")
else:
  print(f"Null Value : {null_count}")

Null Value Verification Successfull


In [ ]:
# 1. Get the combined raw list
raw_all = pd.concat([
    df_enrolment['district'],
    df_demographic['district'],
    df_biometric['district']
])

raw_districts_cleaned = raw_all.apply(ultra_clean_names).unique()
raw_count = len(raw_districts_cleaned)

# 3. Get your master count
cleaned_count = master['district'].nunique()

print(f"District in Raw (Standardized): {raw_count}")
print(f"District in Cleaned: {cleaned_count}")

if raw_count == cleaned_count:
    print("District Coverage Verification Successful")
else:
    # See the remaining differences if any
    master_set = set(master['district'].unique())
    raw_set = set(raw_districts_cleaned)
    print(f"Remaining Difference: {raw_set.symmetric_difference(master_set)}")

District in Raw (Standardized): 50
District in Cleaned: 50
District Coverage Verification Successful


In [ ]:
#SANITY Check(are there negative values)
negative_values = (master.select_dtypes(include=['number']) < 0).sum().sum()

if negative_values == 0:
  print("Verfication passes: No negative valuse")
else:
  print(f"Verification fails: {negative_values} negative values")

Verfication passes: No negative valuse


**FINAL CLEANED FILE**

In [ ]:
#exporting the file
master.to_csv('/content/drive/MyDrive/Data_Hackathon/Cleaned_Dataset/maharastra_master_cleaned.csv', index=False)

print("Master File Created Successfully")
print(f"Total Rows: {len(master)}")
print(f"Districts Found: {master['district'].nunique()}")

Master File Created Successfully
Total Rows: 360345
Districts Found: 50
